In [ ]:
pip install selenium
pip install webdriver_manager
pip install pyquery
pip install jieba
pip install google-cloud-vision
pip install --upgrade google-cloud-storage

In [ ]:
import math, time, random, datetime
from datetime import datetime as dt
import pandas as pd, numpy as np
from pandas.io.json import json_normalize
import os,io,re,json,pdb,warnings
from ast import literal_eval

from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.error import HTTPError
from urllib.request import urlopen
from webdriver_manager.chrome import ChromeDriverManager
import requests
from pyquery import PyQuery as pq
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from collections import Counter
from selenium.webdriver.chrome.options import Options

from google.oauth2 import service_account
from google.cloud import vision
from google.cloud.vision import types

import jieba
import shutil

In [ ]:
class IG:
    def __init__(self):
        pass
    
    #看各文章的like
    def Click_like():
        buttons = driver.find_elements_by_tag_name('button')
        time.sleep(3)
        for j in buttons:
            if j.text[:2] =='其他':
                j.click()
                time.sleep(2)
                break
            elif j.text[-2:] == '個讚':
                j.click()        
                time.sleep(2)
                
    def Login(name):
        login_url = "https://www.instagram.com/accounts/login/?hl=zh-tw&next=%2F"+str(name)+"%2F&source=desktop_nav"
        driver.get(login_url)
        time.sleep(2)
        elem_user = driver.find_element_by_name("username")
        elem_user.clear
        elem_user.send_keys("") ##輸入
        elem_pwd = driver.find_element_by_name("password")
        elem_pwd.clear
        elem_pwd.send_keys("")##輸入
        time.sleep(2)
        buttons = driver.find_elements_by_tag_name('button')
        time.sleep(2)
        buttons[1].click()
        time.sleep(2)
        try:
            buttons = driver.find_elements_by_tag_name('button')
            time.sleep(1)
            for j in buttons:
                if j.text[:2] =='稍後':
                    j.click() 
                    break
        except:
            driver.refresh()
            buttons = driver.find_elements_by_tag_name('button')
            for j in buttons:
                if j.text[:2] =='稍後':
                    j.click() 
                    break
        
        
        time.sleep(2)
        return '登入成功'

    def Nowarnings():
        warnings.filterwarnings('ignore')

    def Allfans(name):
        global n_o_f,followers_data
        time.sleep(1)
        #點開追蹤粉絲
        try:
            
            button=WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"//a[@href='/"+str(name)+"/followers/']")))
            button.click()
            
        except:
            driver.refresh()
            button=WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"//a[@href='/"+str(name)+"/followers/']")))
            button.click()

        nu_of_followers = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="react-root"]/section/main/div/header/section/ul/li[2]/a/span')))
        n_o_f = int(nu_of_followers.text)#總粉絲人數
        
        fBody = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='isgrP']")))
        scroll = 0
        while scroll < n_o_f // 5:
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;', fBody)
            time.sleep(0.5)
            scroll += 1
        fList = driver.find_elements_by_xpath("//div[@class='isgrP']//li")
        user_name = []

        for f in fList:
            s = f.text
            l = s.split('\n')
            user_name.append(l[0])

        followers_data = pd.DataFrame({'USER_NAMES': user_name})
        print('Number of Followers are : ', n_o_f)
        

    def CreatFile(name):
        global dirpath
        dirpath = 'C:\\Users\justi\OneDrive\桌面\IG_ana\\' + name 
        try:
            shutil.rmtree(dirpath)
        except:
            pass
        if not os.path.exists(dirpath):
            os.makedirs(dirpath)

In [ ]:
class  IG_detect:
    def detect_labels_uri(uri):
        
        client = vision.ImageAnnotatorClient()
        image = vision.types.Image()
        image.source.image_uri = uri
        response = client.label_detection(image=image)
        labels = response.label_annotations


    def extract_labels_from_gvision(url):
        my_labels = detect_labels_uri(url)
        my_label_list = list()
        for label in my_labels:
            my_label_list.append(label.description)

        # 觀光景點辨識
    def tourist_attraction(name):
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"inflix-2284c0868282.json"
        taiwan=0
        asia  =0
        world =0
        for i in range(len(links)):
            attraction = 0
            dirpath = 'C:\\Users\justi\OneDrive\桌面\IG_ana\\' + name 
            file_path = dirpath + '\\' + '{}.jpg'.format(i)
            client = vision.ImageAnnotatorClient()
            file_name=file_path
            with io.open(file_name, 'rb') as image_file:
                content = image_file.read()
            image = types.Image(content=content)
            response = client.landmark_detection(image=image)
            landmarks = response.landmark_annotations
            if len(list(landmarks)) >0:
                print('第{}張圖片辨識為:{}:'.format(i,landmarks[0].description))
                attraction+=1
                if  119.36 <landmarks[0].locations[0].lat_lng.longitude< 122.053 and 21.8035 <landmarks[0].locations[0].lat_lng.latitude < 25.4374:
                    taiwan+=1
                    print('地區為台灣')
                elif 69.257131 <landmarks[0].locations[0].lat_lng.longitude < 152.620730 and  -9.995534 <landmarks[0].locations[0].lat_lng.latitude < 45.6214 :
                    asia+=1
                    print('地區為亞洲')
                else:
                    world+=1
                    print('地區為國外')
            
            else:
                pass
        print('\n'+'---圖片辨識結果---')
        print('辨識為台灣共{}張'.format(taiwan))
        print('辨識為亞洲共{}張'.format(asia))
        print('辨識為國外共{}張'.format(world))
        return '辨識出旅遊景點的圖片張數:',attraction

    def NLP(s):
        global positive_ntu_count,negative_ntu_count,life_count,restaurant_count,fintech_count,finance_count,Data_Path
        IG.Nowarnings()
        Data_Path = 'C:\\Users\justi\OneDrive\桌面\IG_ana\\'
        #之後要再分類出兩個excel比較省時間
        df_ntu= pd.read_csv(Data_Path + 'ntusd.csv',encoding='ansi')
        pos_ntu =[]
        neg_ntu=[]
        for i in range(28798):
            if df_ntu.iloc[i,:].p > df_ntu.iloc[i,:].n:
                pos_ntu.append(df_ntu.iloc[i,:].Name)
            elif df_ntu.iloc[i,:].p < df_ntu.iloc[i,:].n:
                neg_ntu.append(df_ntu.iloc[i,:].Name)

        #斷詞
        positive_ntu_count=0
        negative_ntu_count=0
        life_count=0
        life_count_box=[]
        restaurant_count=0
        restaurant_count_box=[]
        fintech_count=0
        fintech_count_box=[]
        finance_count=0
        finance_count_box=[]
        #travel_count=0

        words  = jieba.lcut(s)  
        for s in pos_ntu:
            for t in words:
                if s == t:
                    positive_ntu_count += 1
        for s in neg_ntu:
            for t in words:
                if s == t:
                    negative_ntu_count += 1     
        for s in life:
            for t in words:
                if s == t:
                    life_count += 1
                    life_count_box.append(s)
        for s in restaurant:
            for t in words:
                if s == t:
                    restaurant_count += 1
                    restaurant_count_box.append(s)
        for s in fintech:
            for t in words:
                if s == t:
                    fintech_count += 1
                    fintech_count_box.append(s)
        for s in finance:
            for t in words:
                if s == t:
                    finance_count += 1
                    finance_count_box.append(s)
        print('\n'+'---NLP辨識結果---')
        print('生活詞對應數 :{} 個'.format(life_count))
        if life_count !=0 :
            print('生活詞對應詞彙 : {}'.format(life_count_box))
        print('餐廳詞對應數 :{} 個'.format(restaurant_count))
        if restaurant_count !=0 :
            print('餐廳詞對應詞彙 : {}'.format(restaurant_count_box))
        print('科技詞對應數 :{} 個'.format(fintech_count))
        if fintech_count !=0 :
            print('科技詞對應詞彙 : {}'.format(fintech_count_box))
        print('金融詞對應數 :{} 個'.format(finance_count))
        if finance_count !=0 :
            print('金融詞對應詞彙 : {}'.format(finance_count_box))
        print('正向_ntu詞對應數 {} 個'.format(positive_ntu_count))
        print('負向_ntu詞對應數 {} 個'.format(negative_ntu_count)) 

In [ ]:
def RunIG():
    print('欲分析的帳號ID:')
    name = str(input())
    IG.Nowarnings()
    instagram_crawler(name)
    if post_num > 0:
        IG_detect.NLP(s)
        IG_detect.tourist_attraction(name)
    else:
        print('無文章無法分析')
    print("Running Time: %s" % datetime.timedelta(seconds=end_time))

In [ ]:
#開發內容：關鍵字貼標-全球、財經、數位科技、旅行、美食餐廳、生活

life   = ['實習','結婚','婚禮','Worldgym','健身','貓','狗','寵物',
          '女友','女朋友','男友','男朋友','老婆','老公','擁抱','讀書','看書',
          '打工','公益','浮潛','衝浪','畢業','基督','溫泉','高爾夫球','家人',
          'Netflix','netflix','信義']


travel =['日本','韓國','沖繩','京都','關島','出國','越南','歐洲','泰國','新加坡','首爾','葡萄牙','土耳其','馬來西亞','加拿大',
         '美國','德國','法國','希臘','義大利','摩洛哥','俄羅斯','墨爾本','智利','札幌','西班牙','秘魯','英國','瓜地馬拉',
         '吳哥窟','澳大利亞','澳洲','奧地利','捷克','巴西']


restaurant=['牛排','酒吧','壽司','法式','法國菜','港式']


fintech =['Fintech','AI','人工智慧','數位','科技','區塊鏈','比特幣','跨平台','加密','新創','去中心化','辨識','憑證','金融服務',
          '挖礦','指紋辨識','網銀','供應鏈','虛擬帳','機器學習','爬蟲','深度學習','API','普匯','金融科技','第三方','P2P','網銀',
          '演算法','Python','C++','Java','數據分析']


finance=['財經','財金','投資','基金','股票','投資人','理財','ETF','債券','債權','經濟','股市','資產','收益','證券','資金','利率',
         '投信','股東','台股','期貨','巴菲特','股利','報酬率','標的','營收','儲蓄','配息','股價','財富','原油','交易所','每股',
         '槓桿','股份','券商','定存','存款','配股','獲利','外幣','第一桶金','標普','選股','匯率','信用','財報','個股','總經',
         'NASDAQ','貨幣','貸款','外資','融資','併購']

In [ ]:
def instagram_crawler(name):
    global driver,fans_box,links,s , end_time , avg_fan_per_posts , n_o_f , run_time , effective_posts , post_num
    
    IG.Nowarnings()
    
#     chrome_options = Options() # 啟動無頭模式
#     chrome_options.add_argument('--headless')  #規避google bug
#     chrome_options.add_argument('--disable-gpu')
    fans_box=[]
    #先創建資料夾放圖片
    IG.CreatFile(name)

    start_time = time.time()
    
#     driver = webdriver.Chrome(chrome_options=chrome_options,executable_path='./chromedriver')
    driver = webdriver.Chrome()
    print('開啟網頁成功')
    
    # Login #
    
    IG.Login(name)
    time.sleep(2)
    
    IG.Allfans(name) # 紀錄粉絲數
    
    driver.refresh() # F5重新整理

    wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class=' _2z6nI']")))
    links=[]
    source = driver.page_source
    data=BeautifulSoup(source, 'html.parser')
    body = data.find('body')
    
    post_num = body.find('span',class_='g47SY').text
    #若無貼文則直接顯示 0
    if post_num == 'NaN':
        post_num = 0
    else:
        post_num=int(post_num)

        
    print('Number of post are : ', post_num)
    while len(links) != post_num:
        links=[]
        driver.refresh()
    #
        last_height = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print('網頁更新中...')

        while True:
            source = driver.page_source
            data=BeautifulSoup(source, 'html.parser')
            body = data.find('body')
            time.sleep(1)
            for link in body.findAll('a'):
                if re.match("/p/", link.get('href')):
                    links.append(link.get('href'))
            # 抓到整個瀏覽器高度，然後滑下去
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            # 計算新高度，與舊的高度比一比，若一樣代表到底了
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print('到達頁面底端')
                break
            last_height = new_height
        links = list(set(links))
    #
    #print('共抓取了{}筆資料'.format(len(links)))
    print('----------------------------------------------')
    
    s=''
    print('開始處理內文')
    
    Excluding_posts = 0
    Two_month = 0 
    Four_month = 0
    Six_month = 0
    Eig_month = 0
    Ten_month = 0
    Twe_month = 0
    
    
    
    for i in range(len(links)):

        page_source=driver.get('https://www.instagram.com'+links[i])
        try:
            wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='eo2As ']")))
        except:
            driver.refresh()
        print('開始處理第{}篇'.format(i+1))
        
        html = driver.page_source
        data = BeautifulSoup(html,'html.parser')
        body = data.find('body')
        for link in body.find_all('script'):
            if link.text.strip().startswith('window.__additionalDataLoaded'):
                a=link
        a=a.text.replace(';', '')[:-1][31:].replace(links[i],'')[2:]
        json_data = json.loads(a)
        posts = json_data
        display_url=posts['graphql']['shortcode_media']['display_url'] 
        comments= posts['graphql']['shortcode_media']['edge_media_to_parent_comment']['count']
        
        
        try: 
            s += posts['graphql']['shortcode_media']['edge_media_to_caption']['edges'][0]['node']['text'] #串文字
        except:
            pass
        
        
        #處理圖片
        response = requests.get(display_url)
        if response.status_code == 200:
            content = response.content

        file_path = dirpath + '\\' + '{}.jpg'.format(i)
        with open(file_path,'wb') as handler:
            handler.write(content)
        
        
        
        #只擷取兩年內的文章,讚數
        
        now=datetime.datetime.now()
        t=dt.strptime(body.find_all(class_='_1o9PC Nzb55')[0]['datetime'][:10], "%Y-%m-%d")
        if (now-t).days > 1095: #只擷取三年內的文章,讚數
            Excluding_posts += 1
            time.sleep(1)
            print('讚數不計入')
            continue    
            
        else:
            if (now-t).days <= 60:
                Two_month += 1
            elif 60 < (now-t).days <= 120 :
                Four_month += 1
            elif 120 < (now-t).days <= 180 :
                Six_month += 1
            elif 180 < (now-t).days <= 240 :
                Eig_month += 1
            elif 240 < (now-t).days <= 300 :
                Ten_month += 1
            elif 300 < (now-t).days <= 365 :
                Twe_month += 1

        #點like
        IG.Click_like()
        
        user_name_ = []
        #滾動
        try:
            element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//div[@style='height: 356px; overflow: hidden auto;']")))
            last_height = driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;',element)
            time.sleep(3)
            while True:
                e = driver.find_elements_by_xpath("//div[@style='height: 356px; overflow: hidden auto;']//div[@class='                    Igw0E   rBNOH        eGOV_     ybXk5    _4EzTm                                                                                   XfCBB          HVWg4                 ']")
                for f in e:
                    l = f.text.split('\n')
                    if l[0] not in user_name_:
                        user_name_.append(l[0])
                # 抓到整個瀏覽器高度，然後滑下去
                driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;',element)
                # 等0.5秒
                time.sleep(0.5)
                # 計算新高度，與舊的高度比一比，若一樣代表到底了
                new_height = driver.execute_script("return arguments[0].scrollTop", element)
                if new_height == last_height:
                    break
                last_height = new_height
        except:
            pass

        likes=int(len(user_name_)) 

        #將每次的內文粉絲放到fans_box以便最後統計
        for p in user_name_:
            fans_box.append(p)

        print('共{}位讚數'.format(len(user_name_)))

        
    result=Counter(fans_box)
    rank = []
    for r in result.most_common(15):
        rank.append(r[0])
        
    effective_posts = len(links) - Excluding_posts    
    
    if effective_posts != 0 :
        avg_fan_per_posts = len(fans_box)//effective_posts
        effective_fans_ratio = round((avg_fan_per_posts/n_o_f)*100,0)
    else:
        avg_fan_per_posts = 0
        effective_fans_ratio = 0
    
    
    end_time = (time.time() - start_time)
    

    driver.quit()
    
    print('\n')
    print('用戶名稱 : {}'.format(name))
    print('用戶貼文總數 : {}'.format(len(links)))
    print('用戶有效貼文總數 : {}'.format(effective_posts))
    print('用戶粉絲人數 : {}'.format(n_o_f))
    print('貼文平均按讚數 : {}'.format(avg_fan_per_posts))
    print('活躍粉絲人數比例 : {}%'.format(effective_fans_ratio))
    print('用戶熟識名單前十五名:{}'.format(rank))
    print('2個月內至今發文文章數 : {}'.format(Two_month))
    print('2 ~ 4個月內發文文章數 : {}'.format(Four_month))
    print('4 ~ 6個月內發文文章數 : {}'.format(Six_month))
    print('6 ~ 8內至今發文文章數 : {}'.format(Eig_month))
    print('8 ~ 10個月內發文文章數 : {}'.format(Ten_month))
    print('10 ~ 12個月內發文文章數 : {}'.format(Twe_month))

In [ ]:
RunIG()